In [1]:
#%load_ext autoreload
#%autoreload 2

# Build the MATLAB Executables

- This notebook provides a mechanism to compile the MATLAB executables required by the ghubex3 tool.<br />

- The MATLAB scripts in the ./src directory are compiled on CCR and the returned MATLAB executables are moved to the ./bin directory.<br />

- The MATLAB executables are compiled on CCR using the MATLAB mcc function.<br />

- Click the Run Workflow button to compile the MATLAB executables.<br />

In [2]:
import sys
import os
import getpass
import platform
import shutil
import atexit
import math
import numpy as np
import pandas as pd
import time

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output, Image, Javascript
#import xml.etree.ElementTree as et

import hublib
#print (help(hublib))
import hublib.ui as ui
#print (help(ui))
import hublib.use
#print (help(hublib.use))

# Setup paths to executables
scriptpath = os.path.realpath(" ")
        
# Get the parent dirs
self_tooldir = os.path.dirname(scriptpath)

# Setup path to python and bash scripts
self_srcdir = os.path.join(self_tooldir, "src")

# Setup path to python and bash scripts
self_bindir = os.path.join(self_tooldir, "bin")

# Add to PYTHONPATH
sys.path.insert (1, self_bindir)

# Set up path to the current data directory
self_datadir = os.path.join(self_tooldir, "data")

# Set up path to the current doc directory
self_docdir = os.path.join(self_tooldir, "doc")

# Set up path to the current session directory
self_workingdir = os.getcwd()

# Set up path to the user's home directory
self_homedir = os.path.expanduser("~")

# Initialize the dated run directory.
# Workflow results are not available until after a workflow is executed via Pegasus and completes
self_rundir = ""

self_user = getpass.getuser()

# Version of Pegasus
# Note: when switching the version of Pegasus, delete ~/.pegasus/workflow.db
%use pegasus-5.0.1
from buildWrapper import buildWrapper

widget_border_style = '1px solid black'
widget_output_border_style = '1px solid black'

BOLD = '\033[1m'
SUCCESS = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
END = '\033[0m'

dropdown_str_width = 16

dropdown_width = '965px'
dropdown_height = '30px'
button_width = '250px'
button_height = '40px'
ui_string_width = '96.5%'
ui_dropdown_width = '96.2%'

#print ('os.access(self_tooldir, os.W_OK): ', os.access(self_bindir, os.W_OK))


<IPython.core.display.Javascript object>

In [3]:
#https://stackoverflow.com/questions/36757301/disable-ipython-notebook-autoscrolling

In [4]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [5]:
#https://api.jquery.com/ready/
HTML('''
<script>
    function scroll_to_top() {
        Jupyter.notebook.scroll_to_top();
    } 
    $( window ).on( "load", scroll_to_top() );
</script>
''')

In [6]:
# Button styles
HTML('''
<style>.buttontextclass { color:black ; font-size:130%}</style>
''')

In [7]:
#tooldir = os.path.dirname(os.path.dirname(os.path.realpath(os.path.abspath(__file__))))
#print ('tooldir: ', tooldir)


In [8]:
 # Run Workflow

maxwalltime = ui.Number(
    name = 'Maximum Walltime',
    description = 'Maximum Walltime [min]',
    units = 'min',
    value = '30.0',
    min = '5.0',
    max = '60.0'
)

workflow_run_options_form = ui.Form([maxwalltime], name = 'Workflow Run Options')

def run_workflow(p):
    
    # print (p) #Button
    #global self_workflow_succeeded
    #self_workflow_succeeded = False
        
    workflow_progress.clear_output()
    #workflow_results.clear_output()
        
    with workflow_progress:
        
        runWorkflowButton.disabled = True
        #show_log_output_button.disabled = True
        
        start_time = time.time()
        
        try:
            
            # All MATLAB scripts in the src directory will be copied to vortex for compiling the MATLAB exeutables.
            # The matlabBuild.sh script in the remotebin directory contains specific instructions for compiling the MATLAB executables
            
            # Returned compiled MATLAB executables
            binfiles = ['deg2utm', 'utm2deg']

            for i in range(len(binfiles)):

                binfile = binfiles[i]
                binfilepath = os.path.join(self_bindir, binfile)
                if os.path.exists(binfilepath):
                        print ('removing %s.' %binfilepath)
                        os.remove(binfilepath)
                
            #Note: Workflow execution time depends on the current UB CCR workload.
            print ("Pegasus workflow in progress. This will take approximately 10 minutes...")
            
            #'''
            buildWrapper (" ", \
                self_workingdir, binfiles, int(maxwalltime.value))
            #'''
            
            print ('\nWorkflow elapsed time: ' + str((time.time() - start_time)/60.0) + ' [min]\n')
            
            # Check if the results files were created and transferred from CCR 
            # to determine if workflow completed successfully
            
            workflow_completed_successfully = True
            for i in range (len(binfiles)):
                binfile = binfiles[i]
                binfilepath = os.path.join(self_bindir, binfile)
                if os.path.exists(binfile) == True:
                    shutil.move(binfile, binfilepath)
                else:
                    print ('%s not generated by the workflow.' %binfile)
                    workflow_completed_successfully = False

            if workflow_completed_successfully:

                print ('Workflow completed successfully\n')
                        
            else:

                print ('\nWorkflow did not complete successfully')

                filepath = os.path.join(self_workingdir, 'pegasus.analysis')
                if (os.path.exists(filepath)):
                    print ("pegasus.analysis:\n")
                    f = open(filepath, 'r')
                    output = f.read()
                    f.close()
                    print (output)
       
            #finish_workflow_processing()
        
        except Exception as e:
        
            print ('Workflow Exception: %s\n' %str(e))
       
        runWorkflowButton.disabled = False
        #show_log_output_button.disabled = False


# Abort
# Select Kernel Interrupt
#if self_tW.is_alive() == True:
   #self_tW.terminate()

runWorkflowButton = widgets.Button(description="Run Workflow", disabled=False,\
    layout=widgets.Layout(width=button_width, height=button_height),\
    style= {'button_color':'lightgreen','font_weight':'bold'})
runWorkflowButton.add_class("buttontextclass")
runWorkflowButton.on_click (run_workflow)
#help (runWorkflowButton)

# Note: See /apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/hublib/ui/pathselect.py,
# file property initialized to None, when a file is selected gets set to the selected file.


In [9]:
display(workflow_run_options_form)
display(runWorkflowButton)

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>Workflow Run…

Button(description='Run Workflow', layout=Layout(height='40px', width='250px'), style=ButtonStyle(button_color…

In [10]:
workflow_progress = widgets.Output(layout={'border': '1px solid black'})
display(workflow_progress)

Output(layout=Layout(border='1px solid black'))